In [ ]:
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import traitlets
import datetime
import os
import cv2
import numpy as np
from vision.detector import PlateNumberDetector
from vision.preprocessor import ImagePreprocessor
import subprocess
from SCSCtrl import TTLServo

detector = PlateNumberDetector(
    model="clova",                 # OCR할 엔진 선택 ("paddle" 또는 "clova")
    plate_similarity_thresh=82,     # 타겟 번호판과 일치하는지 판단할 유사도 임계치(기본값 80)
    debug_mode=False,                # 디버그 모드 : 사전 OCR 데이터(data/demo/에 존재) 불러오기 -> API 호출 수 줄이거나 모델 로드 리소스 생략 가능
)

# 이미지 전처리 객체
image_preprocessor = ImagePreprocessor(
    gamma=1.5,                      # 이미지의 감마 값 (1보다 큰 값은 어두운 영역을 더 밝게, 작은 값은 밝은 영역을 더 어둡게 조정)
    clahe_clip_limit=4.0,           # CLAHE 기법 적용 시, 대비 제한 값 (큰 값은 강한 대비를, 작은 값은 약한 대비를 적용)
    contrast_alpha=1.5,             # 전체 이미지의 대비 조정 강도 (1은 변경 없음, 값이 클수록 대비가 강해짐)
    contrast_beta=0                 # 전체 이미지의 밝기 조정 값 (0은 변경 없음, 양수는 밝기 증가, 음수는 감소)
)

# ==========================================
# 1. 카메라 설정 (기존 코드 기반)
# ========================================
# camera_cap=Camera.instance(width=1632, height=1232, capture_width=1632, capture_height=1232)
camera = Camera.instance(width=816, height=616)

image = widgets.Image(format='jpeg', width=224, height=224)
# 카메라 영상과 이미지 위젯 연결
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998
[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)


[System] WebSocket 프로세서 시작


In [ ]:
from jetbot import Robot
import time

robot = Robot()

#모터 제어 함수
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)

def step_backward(change):
    robot.backward(0.4)

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

# ==========================================
# 3. 스냅샷(사진 저장) 기능 구현
# ==========================================
def take_snapshot(change):

    TTLServo.servoAngleCtrl(5, 45, 1, 150)
    #  image_data = np.frombuffer(camera.value, dtype=np.uint8)
    image_data = camera.value
    frame=image_data


    if frame is None:
        print("이미지 디코딩 실패")
    else:
        print("이미지 로드 성공")
    
#     frame = image_preprocessor.gamma_correction(frame)
    result = detector.detect(frame, target='187고1604')
    cv2.imwrite('temp.jpg', frame)
# ==========================================
# 4. 버튼 위젯 생성 및 레이아웃
# ==========================================
# 스타일 설정
layout_btn = widgets.Layout(width='80px', height='40px')
layout_box = widgets.Layout(align_items='center')

# 이동 버튼 생성
btn_fwd = widgets.Button(description='전진 ▲', layout=layout_btn)
btn_back = widgets.Button(description='후진 ▼', layout=layout_btn)
btn_left = widgets.Button(description='좌 ◀', layout=layout_btn)
btn_right = widgets.Button(description='우 ▶', layout=layout_btn)
btn_stop = widgets.Button(description='정지 ■', button_style='danger', layout=layout_btn)

# 스냅샷 버튼 생성 (눈에 잘 띄게 초록색으로 설정)
btn_snap = widgets.Button(
    description='스냅샷 저장 📷', 
    button_style='success', # 'success'는 보통 초록색
    layout=widgets.Layout(width='250px', height='50px', margin='20px 0px 0px 0px')
)

# 버튼 이벤트 연결
btn_fwd.on_click(step_forward)
btn_back.on_click(step_backward)
btn_left.on_click(step_left)
btn_right.on_click(step_right)
btn_stop.on_click(stop)
btn_snap.on_click(take_snapshot)

# ==========================================
# 5. UI 배치 (화면 구성)
# ==========================================
# 방향키 배치 (십자 형태)
#    [전진]
# [좌][정지][우]
#    [후진]
controls = widgets.VBox([
    widgets.HBox([widgets.Label(layout=widgets.Layout(width='80px')), btn_fwd, widgets.Label(layout=widgets.Layout(width='80px'))]),
    widgets.HBox([btn_left, btn_stop, btn_right]),
    widgets.HBox([widgets.Label(layout=widgets.Layout(width='80px')), btn_back, widgets.Label(layout=widgets.Layout(width='80px'))]),
    btn_snap # 컨트롤러 아래에 스냅샷 버튼 배치
], layout=layout_box)

# 전체 화면: [카메라 영상]  [컨트롤 패널]
ui = widgets.HBox([image, widgets.Label(layout=widgets.Layout(width='20px')), controls])

display(ui)

In [ ]:
camera.stop()